# BOSS直聘数据建模

## 1、数据预处理

### 1.1 导入必要的库

In [1]:
# 设置显示所有行的选项
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#解决kaggle中文显示乱码
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

# Path to the custom font
font_path = '/kaggle/input/chinese-fonts/NotoSansSC-VariableFont_wght.ttf'

# Add the custom font to the font manager
font_manager.fontManager.addfont(font_path)

# After adding the font, search for it by filename to get the correct font name
for font in font_manager.fontManager.ttflist:
    if font.fname == font_path:
        print(f"Found font: {font.name}")
        plt.rcParams['font.family'] = font.name
        break

Found font: Noto Sans SC


### 1.2 读取数据文件

In [ ]:
import re
import pandas as pd

# 读取Excel文件
df = pd.read_excel("/kaggle/input/boss-zhipin-sample-data/BOSS_Zhipin_Sample_Data.xlsx")

df = df[:1000]

# 显示前几行数据
print(df.head())

### 1.3 清洗数据
删除“薪资”列中，包含“元/单”“面议”等无法用于分析的数据<br>
删除薪资列为空值的数据行

In [ ]:
# 删除包含“元/单”的行
df = df[~df['薪资'].str.contains('元/单|面议', na=False)]

# 删除 '薪资' 列为空值的行
df.dropna(subset=['薪资'], inplace=True)

# 删除 '工作地点' 列为空值的行
df.dropna(subset=['工作地点'], inplace=True)

# 删除 '工作地点' 列为空值的行
df.dropna(subset=['工作地点'], inplace=True)

# 删除'职位描述'中包含“兼职”的行
#df = df[~df['职位描述'].str.contains('兼职|日结', na=False)]

# 删除'职位描述'中包含“兼职”的行
df = df[~df['薪资'].str.contains('元/时', na=False)]

# 查看删除后的 DataFrame
unique_salaries = df['薪资'].unique()
print(len(df))

### 1.4 转换数据类型

In [ ]:
# 查看各列的唯一值
unique_columns = df['薪资'].unique()
#unique_columns = df['经验要求'].unique()

# 打印结果
print(unique_columns)

#查看后发现，薪资可分为：
# 单薪：20元/单
# 时薪：100元/时
# 日薪：300元/日
# 周薪：800元/周
# 月薪：5-8K，3000-5000元/月
# 年薪：8-10K·15薪

#### 1.4.1 将“经验要求”转换为数值分类，方便建模

In [ ]:
#使用代码查看“经验要求”的唯一值：unique_columns = df['经验要求'].unique()
#“经验要求”的唯一值为：['经验不限' '在校/应届' '1年以内' '1-3年' '3-5年' '5-10年'  '10年以上' nan]
#使用如下数值代表不同分类
#    '经验不限': 0,
#    '在校/应届': 1,
#    '1年以内': 2,
#    '1-3年': 3,
#    '3-5年': 4,
#    '5-10年': 5,
#    '10年以上': 6


# 将经验要求进行分类
import pandas as pd

# 定义映射字典
experience_mapping = {
    '经验不限': 0,
    '在校/应届': 1,
    '1年以内': 2,
    '1-3年': 3,
    '3-5年': 4,
    '5-10年': 5,
    '10年以上': 6
}

# 自定义函数进行分类
def classify_experience(val):
    if pd.isna(val):
        return 0  # 缺失值归为 0
    return experience_mapping.get(val, 0)

# 应用分类函数
df['经验要求分类'] = df['经验要求'].apply(classify_experience)

#### 1.4.2 将“学历要求”转换为数值分类

In [ ]:
#使用代码查看“学历要求”的唯一值：unique_columns = df['学历要求'].unique()
#“学历要求”的唯一值为：['学历不限' '初中及以下' '中专及以下' '中专/中技' '高中' '大专' '本科' '硕士' '博士']
#使用如下数值代表不同分类
#    '学历不限': 0,
#    '初中及以下': 1,
#    '中专及以下': 2,
#    '中专/中技': 3,
#    '高中': 4,
#    '大专': 5,
#    '本科': 6,
#    '硕士': 7,
#    '博士': 8


# 学历要求映射字典
education_mapping = {
    '学历不限': 0,
    '初中及以下': 1,
    '中专及以下': 2,
    '中专/中技': 3,
    '高中': 4,
    '大专': 5,
    '本科': 6,
    '硕士': 7,
    '博士': 8
}

# 分类函数
def classify_education(val):
    return education_mapping[val]

# 应用分类
df['学历要求分类'] = df['学历要求'].apply(classify_education)

#### 1.4.3 将“公司规模”转换为数值分类

In [ ]:
#使用代码查看“公司规模”的唯一值：unique_columns = df['公司规模'].unique()
#“公司规模”的唯一值为：['0-20人' '100-499人' '500-999人' '1000-9999人' '20-99人' '10000人以上']
#使用如下数值代表不同分类
#    '0-20人': 0,
#    '20-99人': 1,
#    '100-499人': 2,
#    '500-999人': 3,
#    '1000-9999人': 4,
#    '10000人以上': 5


# 公司规模映射
scale_mapping = {
    '0-20人': 0,
    '20-99人': 1,
    '100-499人': 2,
    '500-999人': 3,
    '1000-9999人': 4,
    '10000人以上': 5
}

#删除公司规模为’nan‘的行
df.dropna(subset=['公司规模'], inplace=True)

# 分类函数
def classify_company_size(val):
    return scale_mapping[val]

#### 1.4.5 将薪资转换为数值对象
我们需要将薪资转换成“月薪下界”，“月薪上界”，“月薪中值”，“月薪区间宽度”，便于建模

In [ ]:
#统一薪资格式

import pandas as pd
import re

def parse_salary_to_monthly(salary_str):
    salary_str = str(salary_str).strip()
    
    # 匹配时薪（元/时）
    match = re.match(r'(\d+)-(\d+)\s*元/时', salary_str)
    if match:
        min_s, max_s = float(match[1]), float(match[2])
        min_month = min_s * 8 * 22
        max_month = max_s * 8 * 22
        return min_month, max_month

    # 匹配日薪（元/天）
    match = re.match(r'(\d+)-(\d+)\s*元/天', salary_str)
    if match:
        min_s, max_s = float(match[1]), float(match[2])
        min_month = min_s * 22
        max_month = max_s * 22
        return min_month, max_month

    # 匹配周薪（元/周）
    match = re.match(r'(\d+)-(\d+)\s*元/周', salary_str)
    if match:
        min_s, max_s = float(match[1]), float(match[2])
        min_month = min_s * 4
        max_month = max_s * 4
        return min_month, max_month

    # 匹配月薪（元/月）
    match = re.match(r'(\d+)-(\d+)\s*元/月', salary_str)
    if match:
        min_s, max_s = float(match[1]), float(match[2])
        return min_s, max_s

    # 匹配K薪形式（7-10K·n薪）
    match = re.match(r'(\d+)-(\d+)K·(\d+)薪', salary_str)
    if match:
        min_s, max_s, n = float(match[1]), float(match[2]), float(match[3])
        min_month = min_s * 1000 * n / 12
        max_month = max_s * 1000 * n / 12
        return round(min_month), round(max_month)

    # 匹配纯K（18-19K）
    match = re.match(r'(\d+)-(\d+)K', salary_str)
    if match:
        min_s, max_s = float(match[1]) * 1000, float(match[2]) * 1000
        return min_s, max_s

    return None, None  # 无法解析

# 假设你的df中列名为 salary
df[['月薪下界', '月薪上界']] = df['薪资'].apply(lambda x: pd.Series(parse_salary_to_monthly(x)))

# 派生中位数和区间宽度
df['月薪中值'] = (df['月薪下界'] + df['月薪上界']) / 2
df['月薪区间宽度'] = round(df['月薪上界'] - df['月薪下界'])

#### 1.4.6 将薪资转换为x-yK的形式，便于绘制云图

In [ ]:
# 将薪资统一为x-yK的形式便于绘制云图

def hourly_to_monthly(salary):
    """将时薪转换为月薪（K）。"""
    if isinstance(salary, str) and '元/时' in salary:
        try:
            # 使用正则表达式提取数字
            numbers = re.findall(r'\d+', salary)
            if len(numbers) == 1:
                hourly_rate = int(numbers[0])
                monthly_rate = hourly_rate * 8 * 22 / 1000  # 计算月薪并转换为 K
                return f'{monthly_rate:.0f}-{monthly_rate:.0f}K'
            elif len(numbers) == 2:
                hourly_rate_min = int(numbers[0])
                hourly_rate_max = int(numbers[1])
                monthly_rate_min = hourly_rate_min * 8 * 22 / 1000
                monthly_rate_max = hourly_rate_max * 8 * 22 / 1000
                return f'{monthly_rate_min:.0f}-{monthly_rate_max:.0f}K'
            else:
                return salary  # 如果无法提取数字，则返回原始薪资
        except ValueError:
            return salary  # 如果转换失败，则返回原始薪资
    else:
        return salary  # 如果不是时薪，则返回原始薪资


def daily_yuan_to_monthly_k(salary):
    """将“xxx-yyy元/天”类型的日薪转换为“x-yK”的月薪形式。"""
    if isinstance(salary, str) and '元/天' in salary:
        try:
            # 使用正则表达式提取数字
            numbers = re.findall(r'\d+', salary)
            if len(numbers) == 2:
                daily_rate_min = int(numbers[0])
                daily_rate_max = int(numbers[1])
                # 假设每月工作 22 天
                monthly_rate_min = daily_rate_min * 22 / 1000
                monthly_rate_max = daily_rate_max * 22 / 1000
                return f'{monthly_rate_min:.0f}-{monthly_rate_max:.0f}K'
            else:
                return salary  # 如果无法提取数字，则返回原始薪资
        except ValueError:
            return salary  # 如果转换失败，则返回原始薪资
    else:
        return salary  # 如果不是“xxx-yyy元/天”类型，则返回原始薪资


# 将周薪转换为月薪
def weekly_to_monthly(salary):
    """将周薪转换为月薪（K）。"""
    if isinstance(salary, str) and '元/周' in salary:
        try:
            # 使用正则表达式提取数字
            numbers = re.findall(r'\d+', salary)
            if len(numbers) == 1:
                weekly_rate = int(numbers[0])
                monthly_rate = weekly_rate * 4 / 1000  # 计算月薪并转换为 K
                return f'{monthly_rate:.0f}-{monthly_rate:.0f}K'
            elif len(numbers) == 2:
                weekly_rate_min = int(numbers[0])
                weekly_rate_max = int(numbers[1])
                monthly_rate_min = weekly_rate_min * 4 / 1000
                monthly_rate_max = weekly_rate_max * 4 / 1000
                return f'{monthly_rate_min:.0f}-{monthly_rate_max:.0f}K'
            else:
                return salary  # 如果无法提取数字，则返回原始薪资
        except ValueError:
            return salary  # 如果转换失败，则返回原始薪资
    else:
        return salary  # 如果不是周薪，则返回原始薪资


def monthly_yuan_to_k(salary):
    """将“xxxx-yyyy元/月”类型的月薪转换为“x-yK”形式。"""
    if isinstance(salary, str) and '元/月' in salary:
        try:
            # 使用正则表达式提取数字
            numbers = re.findall(r'\d+', salary)
            if len(numbers) == 2:
                monthly_rate_min = int(numbers[0]) / 1000
                monthly_rate_max = int(numbers[1]) / 1000
                return f'{monthly_rate_min:.0f}-{monthly_rate_max:.0f}K'
            else:
                return salary  # 如果无法提取数字，则返回原始薪资
        except ValueError:
            return salary  # 如果转换失败，则返回原始薪资
    else:
        return salary  # 如果不是“xxxx-yyyy元/月”类型，则返回原始薪资


def annual_to_monthly_k(salary):
    """将“x-yK·n薪”类型的年薪转换为“x-yK”的月薪形式。"""
    if isinstance(salary, str) and 'K·' in salary:
        try:
            # 使用正则表达式提取数字
            numbers = re.findall(r'\d+', salary)
            if len(numbers) >= 3:
                annual_rate_min = int(numbers[0])
                annual_rate_max = int(numbers[1])
                n = int(numbers[2]) # 获取 'n' 的值
                monthly_rate_min = annual_rate_min * n / 12
                monthly_rate_max = annual_rate_max * n / 12
                return f'{monthly_rate_min:.0f}-{monthly_rate_max:.0f}K'
            else:
                return salary  # 如果无法提取数字，则返回原始薪资
        except ValueError:
            return salary  # 如果转换失败，则返回原始薪资
    else:
        return salary  # 如果不是“x-yK·n薪”类型，则返回原始薪资

# 应用转换函数
df['薪资（K形式）'] = df['薪资'].apply(hourly_to_monthly)
# 应用转换函数
df['薪资（K形式）'] = df['薪资'].apply(daily_yuan_to_monthly_k)
# 应用转换函数
df['薪资（K形式）'] = df['薪资'].apply(weekly_to_monthly)
# 应用转换函数
df['薪资（K形式）'] = df['薪资'].apply(monthly_yuan_to_k)
# 应用转换函数
df['薪资（K形式）'] = df['薪资'].apply(annual_to_monthly_k)

#保存数据预处理之后的excel文件，便于查看
df.to_excel('/kaggle/working//output.xlsx')

### 1.5 修改“经验要求”列
使“经验要求”列的值为：['经验不限' '在校/应届' '1年以内' '1-3年' '3-5年' '5-10年'  '10年以上']，去除其他不规范的值。

In [ ]:
# 定义“经验要求”列表
experience_list = [
    '经验不限',
    '在校/应届',
    '1年以内',
    '1-3年',
    '3-5年',
    '5-10年',
    '10年以上'
]

# 自定义函数进行分类
def classify_experience(val):
    if val in experience_list:
        return val  # 缺失值归为 0
    else:
        return '经验不限'

# 应用分类函数
df['经验要求调整后'] = df['经验要求'].apply(classify_experience)

## 2、构建数据集

### 2.1 构建一个新的DataFrame对象ds
这个DataFrame对象ds包含文本列"text"，月薪下界"lower_salary"，月薪上届"upper_salary"<br>
其中"text"列的文本包含："工作地点","经验要求","学历要求","公司规模"，"公司行业"，"职位描述"等信息

In [ ]:
# 定义一个函数，用于将 DataFrame 的指定列合并为一个新的 "text" 列

def merge_with_titles(row):
    text_parts = []
    text_parts.append(f"工作地点:{row['工作地点']}")
    text_parts.append(f"经验要求:{row['经验要求调整后']}")
    text_parts.append(f"学历要求:{row['学历要求']}")
    text_parts.append(f"公司规模:{row['公司规模']}")
    text_parts.append(f"公司行业:{row['公司行业']}")
    text_parts.append(f"职位描述:{row['职位描述']}")
    return ' '.join(text_parts)

# 创建一个新的空 DataFrame 'ds'
ds = pd.DataFrame({})
ds["text"] = df.apply(merge_with_titles, axis=1)
ds["lower_salary"] = df['月薪下界']
ds["upper_salary"] = df['月薪上界']


### 2.2 检查训练数据的分布情况
在开始训练模型之前，我们要仔细检查数据集，观察数据集的分布，以便获得更好的训练结果。

#### 2.2.1 查看“职位描述”文本长度的分布

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # 用于生成示例数据


# 计算“职位描述”列中每个字符串的长度
df['描述长度'] = df['职位描述'].str.len()

# 查看描述长度的分布情况
print("职位描述字符串长度的分布:")
print(df['描述长度'].describe())
# --- 使用 seaborn 绘制直方图 ---


# 3. 如果数据极度偏斜，可以尝试在X轴上使用对数刻

# 4. 绘制密度直方图 (Y轴显示密度而不是频数)
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='描述长度', bins=100)
plt.title('描述长度密度分布图')
plt.xlabel('描述长度')
#plt.xlim(0, 1200)  # 设置 x 轴范围
plt.ylabel('频数') # Y轴标签变为密度
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### 2.2.2 查看“薪资”的分布情况

In [ ]:
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from matplotlib.gridspec import GridSpec

# 准备 histogram 数据
lower_counts, lower_bins = np.histogram(ds['lower_salary'], bins=50)
upper_counts, upper_bins = np.histogram(ds['upper_salary'], bins=50)

# 对列表进行排序
sorted_lower = sorted(lower_counts)

# 获取第二大的值（倒数第二个元素）
second_lower = sorted_lower[-2]

# 对列表进行排序
sorted_upper = sorted(upper_counts)

# 获取第二大的值（倒数第二个元素）
second_upper = sorted_upper[-2]


# 创建一个总 Figure，并使用 GridSpec 设置 2 行布局
fig = plt.figure(figsize=(16, 16))
gs = GridSpec(2, 1, figure=fig)

# 第一张图：lower_salary
bax1 = brokenaxes(ylims=((0, second_lower+20), (max(lower_counts)-2000, max(lower_counts)+2000)), hspace=0.05,
                  subplot_spec=gs[0])
bax1.hist(lower_bins[:-1], lower_bins, weights=lower_counts,
          color='skyblue', edgecolor='black')
bax1.set_xlabel('月薪下界(元)', fontsize=20)
bax1.set_ylabel('频数', fontsize=20, labelpad=56)
bax1.tick_params(axis='x', labelsize=16)  # 设置 x 轴刻度标签的字体大小
bax1.tick_params(axis='y', labelsize=16)  # 设置 y 轴刻度标签的字体大小
bax1.set_title('月薪下界分布', fontsize=22)

# 第二张图：upper_salary
bax2 = brokenaxes(ylims=((0, second_upper+20), (max(upper_counts)-2000, max(upper_counts)+2000)), hspace=0.05,
                  subplot_spec=gs[1])
bax2.hist(upper_bins[:-1], upper_bins, weights=upper_counts,
          color='lightgreen', edgecolor='black')
bax2.set_xlabel('月薪上界(元)', fontsize=20, labelpad=26)
bax2.set_ylabel('频数', fontsize=20, labelpad=56)
bax2.tick_params(axis='x', labelsize=16)  # 设置 x 轴刻度标签的字体大小
bax2.tick_params(axis='y', labelsize=16)  # 设置 y 轴刻度标签的字体大小
bax2.set_title('月薪上界分布', fontsize=22)
plt.suptitle('分位数法剔除极端值前的薪资分布图', fontsize=24, y=1) 
plt.tight_layout()
plt.savefig("分位数法剔除极端值前的薪资分布图.png", dpi=300)  # dpi 设置图像分辨率
plt.show()

In [ ]:
print("Histogram lower_counts:")

print(lower_counts)
print(lower_bins)

print("Histogram upper_counts:")

print(upper_counts)
print(upper_bins)

### 2.3 对数据进行删除极端值和归一化处理

#### 2.3.1 删除极端值

In [ ]:
# 将薪资大于0.999的值设置为0.999分位值

#设置分位值
lower_percentile = 0.001
upper_percentile = 0.999

def cap_salary(series):
    lower_bound = series.quantile(lower_percentile)
    upper_bound = series.quantile(upper_percentile)
    return series.clip(lower=lower_bound, upper=upper_bound)

ds['lower_salary'] = cap_salary(ds['lower_salary'])
print(f"\nlower_salary 列的 {lower_percentile * 100}% 分位数: {ds['lower_salary'].quantile(lower_percentile)}")
print(f"lower_salary 列的 {upper_percentile * 100}% 分位数: {ds['lower_salary'].quantile(upper_percentile)}")

ds['upper_salary'] = cap_salary(ds['upper_salary'])
print(f"\nupper_salary 列的 {lower_percentile * 100}% 分位数: {ds['upper_salary'].quantile(lower_percentile)}")
print(f"upper_salary 列的 {upper_percentile * 100}% 分位数: {ds['upper_salary'].quantile(upper_percentile)}")

ds['salary'] = list(zip(ds['lower_salary'], ds['upper_salary']))

In [ ]:
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from matplotlib.gridspec import GridSpec

# 准备 histogram 数据
lower_counts, lower_bins = np.histogram(ds['lower_salary'], bins=50)
upper_counts, upper_bins = np.histogram(ds['upper_salary'], bins=50)

# 对列表进行排序
sorted_lower = sorted(lower_counts)

# 获取第二大的值（倒数第二个元素）
second_lower = sorted_lower[-2]

# 对列表进行排序
sorted_upper = sorted(upper_counts)

# 获取第二大的值（倒数第二个元素）
second_upper = sorted_upper[-2]


# 创建一个总 Figure，并使用 GridSpec 设置 2 行布局
fig = plt.figure(figsize=(16, 16))
gs = GridSpec(2, 1, figure=fig)

# 第一张图：lower_salary
bax1 = brokenaxes(ylims=((0, second_lower+20), (max(lower_counts)-2000, max(lower_counts)+2000)), hspace=0.05,
                  subplot_spec=gs[0])
bax1.hist(lower_bins[:-1], lower_bins, weights=lower_counts,
          color='skyblue', edgecolor='black')
bax1.set_xlabel('月薪下界(元)', fontsize=20)
bax1.set_ylabel('频数', fontsize=20, labelpad=56)
bax1.tick_params(axis='x', labelsize=16)  # 设置 x 轴刻度标签的字体大小
bax1.tick_params(axis='y', labelsize=16)  # 设置 y 轴刻度标签的字体大小
bax1.set_title('月薪下界分布', fontsize=22)

# 第二张图：upper_salary
bax2 = brokenaxes(ylims=((0, second_upper+20), (max(upper_counts)-2000, max(upper_counts)+2000)), hspace=0.05,
                  subplot_spec=gs[1])
bax2.hist(upper_bins[:-1], upper_bins, weights=upper_counts,
          color='lightgreen', edgecolor='black')
bax2.set_xlabel('月薪上界(元)', fontsize=20, labelpad=26)
bax2.set_ylabel('频数', fontsize=20, labelpad=56)
bax2.tick_params(axis='x', labelsize=16)  # 设置 x 轴刻度标签的字体大小
bax2.tick_params(axis='y', labelsize=16)  # 设置 y 轴刻度标签的字体大小
bax2.set_title('月薪上界分布', fontsize=22)
plt.suptitle('分位数法剔除极端值后的薪资分布图', fontsize=24, y=1) 
plt.tight_layout()
plt.savefig("分位数法剔除极端值后的薪资分布图.png", dpi=300)  # dpi 设置图像分辨率
plt.show()

In [ ]:
# 打印结果 DataFrame ds
print(ds['lower_salary'].describe())
print(ds['upper_salary'].describe())

#### 2.3.2 对“薪资”进行归一化

In [ ]:
import numpy as np

def max_min_normalize(data, data_min=None, data_max=None):
    """
    将数据归一化到 [0, 1] 区间。

    参数:
        data: 输入数据（可以是 list、numpy 数组、pandas Series 等）
        data_min: 最小值（可选，不提供则使用 data 本身的最小值）
        data_max: 最大值（可选，不提供则使用 data 本身的最大值）

    返回:
        norm_data: 归一化后的数据
        data_min: 最小值（用于逆归一化）
        data_max: 最大值（用于逆归一化）
    """
    data = np.array(data)
    if data_min is None:
        data_min = data.min()
    if data_max is None:
        data_max = data.max()
    norm_data = (data - data_min) / (data_max - data_min + 1e-8)
    return norm_data, data_min, data_max

def max_min_denormalize(norm_data, data_min, data_max):
    """
    将归一化后的数据还原为原始数据。

    参数:
        norm_data: 归一化数据
        data_min: 原始最小值
        data_max: 原始最大值

    返回:
        原始数据
    """
    norm_data = np.array(norm_data)
    original_data = norm_data * (data_max - data_min + 1e-8) + data_min
    return original_data

lower_salary_norm_data, lower_salary_data_min, lower_salary_data_max = max_min_normalize(ds["lower_salary"])
ds["lower_salary_norm"] = lower_salary_norm_data

upper_salary_norm_data, upper_salary_data_min, upper_salary_data_max = max_min_normalize(ds["upper_salary"])
ds["upper_salary_norm"] = upper_salary_norm_data
ds['labels'] = list(zip(ds['lower_salary_norm'], ds['upper_salary_norm']))
ds.head()
print(f"薪资下界的最小值：{lower_salary_data_min}, 薪资下界的最大值：{lower_salary_data_max}")
print(f"薪资上界的最小值：{upper_salary_data_min}, 薪资上界的最大值：{upper_salary_data_max}")

#### 2.3.3 使用KMeans对“薪资”进行聚类，找出6组具有代表性的薪资。

In [ ]:
#聚类
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# 假设 df['salary'] 是包含元组 (min, max) 的列
# Step 1：计算薪资中点
ds['salary_mid'] = ds['salary'].apply(lambda x: (x[0] + x[1]) / 2)

# Step 2：KMeans 聚类为 6 组
X = ds['salary_mid'].values.reshape(-1, 1)
kmeans = KMeans(n_clusters=6, random_state=0).fit(X)

# 聚类标签
ds['cluster'] = kmeans.labels_

# Step 3：对每个簇，找出最接近中心点的原始 salary 区间作为“代表标签”
cluster_centers = kmeans.cluster_centers_.flatten()
representative_labels = {}

for i in range(6):
    cluster_data = ds[ds['cluster'] == i]
    center = cluster_centers[i]
    # 找离中心最近的中点对应的区间
    closest_idx = (cluster_data['salary_mid'] - center).abs().idxmin()
    representative_labels[i] = ds.loc[closest_idx, 'salary']

# Step 4：为每一行分配对应的代表性标签
ds['salary_label'] = ds['cluster']
#ds['salary_label'] = ds['cluster'].map(representative_labels)

# 可选：只保留你需要的列
ds = ds.drop(columns=['salary_mid', 'cluster'])

# 查看结果
print(ds.head(10))
print(representative_labels)
ds.to_excel("ds.xlsx", index=False) 

### 2.4 将DataFrame对象ds载入为transformers 库中的数据集对象dataset

In [ ]:
# 安装相应依赖
import subprocess

#transformers_cmd = "pip install datasets[audio] ipywidgets transformers datasets umap-learn".split()
#process_scatter = subprocess.run(
#    transformers_cmd,
#    shell=True
#    #stdout=subprocess.PIPE,
#    #stderr=subprocess.PIPE,
#)
#transformers_cmd = "apt install git-lfs".split()
#process_scatter = subprocess.run(
#    transformers_cmd,
#    stdout=subprocess.PIPE,
#    stderr=subprocess.PIPE,
#)

#!pip install datasets[audio]
#!pip install datasets[audio]==1.16.1
#!pip install matplotlib
#!pip install ipywidgets
#!pip install pandas==1.5.3 

#!apt install git-lfs

#!pip install transformers datasets
#!pip install transformers==4.13.0 datasets==2.8.0

# Chapter 2 - Classification
#!pip install umap-learn
#!pip install umap-learn==0.5.1

### 2.3 划分训练集、验证集，比例为（90%，10%）

In [ ]:
from datasets import Dataset, DatasetDict

full_dataset = Dataset.from_pandas(ds)
# 先划分出 90% 的训练，10% 剩下的作为验证
split = full_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]  # 这个是验证集（10%）

# 组成一个完整的 DatasetDict（方便后续使用）
dataset_dict = DatasetDict({
    "train": train_dataset,
    "eval": eval_dataset
})
eval_dataset

## 3、将文本编码为token

### 3.1 初始化预训练模型的tokenizer

In [ ]:
from transformers import AutoTokenizer

#谷歌中文bert模型
#model_ckpt = "google-bert/bert-base-chinese"

#哈工大讯飞联合研发的中文bert-wwm模型
model_ckpt = "hfl/chinese-bert-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

### 3.2 定义一个 `tokenize` 函数，将输入文本转换为对应的 `input_ids`（即标记化后的输入序列）。

In [ ]:
def tokenize(batch):
    #return tokenizer(batch["text"], padding=True, truncation=True)
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

#将训练集的前2个文本转换为token，并查看结果
print(tokenize(dataset_dict["train"][:2]))

### 3.3 将数据集的"text"列转换为token

In [ ]:
job_desc_encoded = dataset_dict.map(tokenize, batched=True, batch_size=512)

查看转换后的数据集的列名

In [ ]:
print(job_desc_encoded["train"].column_names)

## 4、将bert预训练模型作为文本特征提取器，提取最后一层隐藏状态(last hidden states)

### 4.1 通过transformers库载入bert预训练模型

In [ ]:
from transformers import AutoModel
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModel.from_pretrained(model_ckpt)

if torch.cuda.device_count() > 1:
    print("使用多 GPU！")
    model = nn.DataParallel(model)

model = model.to(device)

### 4.2 将测试文本转换为token之后传入模型，提取并查看最后一层隐藏状态(hidden states)

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")

In [ ]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

In [ ]:
#查看隐藏状态(hidden states)的维度信息
#1表示批尺寸，6表示共6个token，'CLS' 'this' 'is' 'a' 'test' 'SEP','CLS'表示开头，'SEP'表示结尾
#bert预训练模型提取最后一层隐藏状态为768维向量
outputs.last_hidden_state.size()

### 4.3 定义提取最后一层隐藏状态的函数

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
job_desc_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "labels"])

### 4.4 将训练数据集(包含训练集和验证集)中的所有"text"的token传入模型，提取最后一层隐藏状态

In [ ]:
job_desc_hidden = job_desc_encoded.map(extract_hidden_states, batched=True, batch_size=256)

### 4.5 准备训练和验证数据，并查看数据集的维度

In [ ]:
import numpy as np

X_train = np.array(job_desc_hidden["train"]["hidden_state"])
X_valid = np.array(job_desc_hidden["eval"]["hidden_state"])
y_train = np.array(job_desc_hidden["train"]["labels"])
y_valid = np.array(job_desc_hidden["eval"]["labels"])
X_lebal = np.array(job_desc_hidden["train"]["salary_label"])
X_train.shape, X_valid.shape

### 4.6 训练线性回归模型
将提取的最后一层隐藏状态，与归一化之后的薪资传入线性回归模型，并查看训练好的线性回归模型对验证集的拟合优度

In [ ]:
from sklearn.linear_model import LinearRegression

# We increase `max_iter` to guarantee convergence
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
lr_reg.score(X_valid, y_valid)

### 4.7 训练xgboost模型
将提取的最后一层隐藏状态，与归一化之后的薪资传入xgboost模型（eXtreme Gradient Boosting极限梯度提升），并查看训练好的模型对验证集的拟合优度

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 创建 XGBoost 回归模型
xgbr = xgb.XGBRegressor(objective='reg:squarederror',  # 指定回归任务的损失函数
                        n_estimators=100,             # 树的数量
                        learning_rate=0.1,             # 学习率
                        max_depth=3,                   # 树的最大深度
                        random_state=42)               # 设置随机种子以保证结果的可重复性

# 训练模型
xgbr.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = xgbr.predict(X_valid)

# 评估模型性能（例如，使用均方误差）
mse = mean_squared_error(y_valid, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
#rf.fit(X_train, y_train)
#print("R^2:", rf.score(X_valid, y_valid))

## 5、可视化分析最后一层隐藏状态

最后一层隐藏状态(last hidden states)为768维向量，使用 UMAP（Uniform Manifold Approximation and Projection）对高维数据进行 降维可视化，并结合标签绘制二维空间中的分布情况。
metric="cosine"，使用余弦相似度计算样本之间的距离，适合用于BERT 模型输出的文本高维特征。
X_scaled = MinMaxScaler().fit_transform(X_train)，将特征缩放到 [0, 1] 区间，适合 UMAP 使用。

### 5.1 使用umap将高维信息降维

In [ ]:
!pip install umap-learn
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# Scale features to [0,1] range
X_scaled = MinMaxScaler().fit_transform(X_train)
# Initialize and fit UMAP
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# Create a DataFrame of 2D embeddings
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = X_lebal.tolist()
df_emb.head()

### 5.2 绘制与6个代表性薪资相对应"职位描述"文本的隐含向量分布图

In [ ]:
#解决matplotlib中文显示乱码
import matplotlib

# 设置中文字体（以 Windows 为例，SimHei 是黑体）
matplotlib.rcParams['font.family'] = 'SimHei'

# 避免负号 '-' 显示为方块
matplotlib.rcParams['axes.unicode_minus'] = False

import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(7,5))
axes = axes.flatten()

cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]

for i, cmap in enumerate(cmaps):
    df_emb_sub = df_emb.query(f"label == {i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap,
                   gridsize=20, linewidths=(0,))
    title = f"薪资区间{int(representative_labels[i][0])}-{int(representative_labels[i][1])}元"
    axes[i].set_title(title)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

如上图所示，bert模型所提取的文本特征中，不同薪资对应"职位描述"文本的隐含信息有显著重叠，因此使用bert作为文本特征提取器效果并不理想。

## 6、微调bert模型

### 6.1 使用transformers模型载入bert模型

使用transformers 库，基于bert预训练模型构建一个双变量回归任务（即输出两个连续值），来预测薪资的下界和上界，并使用 Trainer API 对bert预训练模型进行微调。包括模型加载、回归任务配置、RMSE 定义损失函数为均方差函数(Mean Squared Error)、训练参数设置和启动训练过程。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=2,        # 表示回归输出是2个连续值
    problem_type="regression"  # 告诉模型是回归任务（不是分类）
).to(device)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    return {"rmse": rmse}

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(job_desc_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=1,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  #evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error",
                                  report_to=[])

from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=job_desc_encoded["train"],
                  eval_dataset=job_desc_encoded["eval"],
                  tokenizer=tokenizer)
trainer.train();

### 6.2 使用微调后的bert模型，根据"职位描述"文本进行薪资预测
输出为归一化之后的薪资下界和薪资上届

In [ ]:
preds_output = trainer.predict(job_desc_encoded["eval"])

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
print(preds_output)

### 6.3 定义预测函数
预测函数根据"职位描述"的文本，预测薪资范围

In [ ]:
text = "跑钢厂矿山, 据市场营销计划，完成部门销售指标, 具备一定的销售工作经验"
def predict_salary(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    numpy_array = logits.cpu().numpy()[0]
    pred_lower_salary = max_min_denormalize(numpy_array[0], lower_salary_data_min, lower_salary_data_max)
    pred_upper_salary = max_min_denormalize(numpy_array[1], upper_salary_data_min, upper_salary_data_max)
    print(f"预测薪资：{round(pred_lower_salary)}-{round(pred_upper_salary)}元")
    return pred_lower_salary, pred_upper_salary

pred_lower_salary, pred_upper_salary = predict_salary(text)

In [ ]:
lower_salary = max_min_denormalize(0.06, lower_salary_data_min, lower_salary_data_max)
print(int(lower_salary))

### 6.4 选取验证集中的"职位描述"，预测相应的薪资

In [ ]:
x = 0
description = eval_dataset['text'][x]
print(f"工作描述：{description}")
predict_salary(eval_dataset['text'][x])
l_salary = eval_dataset['lower_salary'][x]
u_salary = eval_dataset['upper_salary'][x]

print(f'实际招聘薪资：{l_salary}, {u_salary}')

In [ ]:
# 获取预测结果
preds_output = trainer.predict(job_desc_encoded["eval"])

# 从输出中提取预测值
pred_salary_ranges = []
preds_salary = preds_output.predictions.tolist()
for x in preds_salary:
    pred_lower_salary = max_min_denormalize(x[0], lower_salary_data_min, lower_salary_data_max)
    pred_upper_salary = max_min_denormalize(x[1], upper_salary_data_min, upper_salary_data_max)
    pred_salary_ranges.append((int(pred_lower_salary),int(pred_upper_salary)))

#preds_salary = map(predict_salary, preds_output.predictions.tolist())

true_salary_ranges = job_desc_encoded["eval"]['salary']

In [ ]:
import matplotlib.pyplot as plt

pred_salary_ranges = pred_salary_ranges[:10]
true_salary_ranges = true_salary_ranges[:10]

# 样本数量
n = len(true_salary_ranges)
x = range(n)  # x轴：样本编号

fig, ax = plt.subplots(figsize=(10, 6))

for i in x:
    # 真实薪资范围（蓝色线）
    ax.plot([i, i], true_salary_ranges[i], color='blue', linewidth=4, label='True salary' if i == 0 else "")
    
    # 预测薪资范围（红色线）
    ax.plot([i + 0.1, i + 0.1], pred_salary_ranges[i], color='red', linewidth=4, label='Predicted salary' if i == 0 else "")

# 添加图例和标签
ax.set_xticks([i + 0.1 for i in x])
ax.set_xticklabels([f"Job {i+1}" for i in x])
ax.set_ylabel("Salary (CNY)")
ax.set_title("True vs Predicted Salary Ranges")
ax.legend()
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

### 6.5 选取验证集中的数据，计算预测的薪资与实际薪资对比，并进行可视化

计算预测结果

In [ ]:
import pandas as pd
from torch.nn import MSELoss
import matplotlib.pyplot as plt


loss_fn = MSELoss()

# 获取预测结果
preds_output = trainer.predict(job_desc_encoded["eval"])
# 获取真实薪资
true_salary_ranges = job_desc_encoded["eval"]['salary']

# 提取预测值并反归一化
preds_salary = preds_output.predictions  # 这是一个 NumPy 数组，形状为 (样本数, 2)

# 初始化列表以存储反归一化后的薪资
pred_salary_ranges = []
losses = []

# 遍历预测结果并进行反归一化
for ix, x in enumerate(preds_salary):
    predictions = list((max_min_denormalize(x[0], lower_salary_data_min, lower_salary_data_max),max_min_denormalize(x[1], lower_salary_data_min, lower_salary_data_max)))
    predictions = [int(item) for item in predictions]
    pred_salary_ranges.append(predictions)

    true_tensor = torch.tensor(true_salary_ranges, dtype=torch.float32)
    pred_tensor = torch.tensor(predictions, dtype=torch.float32)

    # 计算损失
    loss = loss_fn(pred_tensor, true_tensor[ix])
    losses.append(loss.item())
    
# 构建 DataFrame
r = pd.DataFrame({
    'pred_salary_ranges': pred_salary_ranges,
    'true_salary_ranges': true_salary_ranges,
    'loss': losses  # 如果有实际的损失值，可以替换这里的 0.1
})

# 显示前几行结果
#print(r.head(20))

绘制对比图

In [ ]:
r = r.sort_values(by='loss', ascending=True)
#随机选取10行
#r = r.sample(n=10, random_state=3)

pred_salary_ranges = r['pred_salary_ranges'].tolist()
true_salary_ranges = r['true_salary_ranges'].tolist()

# 样本数量
n = len(true_salary_ranges)
x = range(n)  # x轴：样本编号

fig, ax = plt.subplots(figsize=(10, 6))

for i in x:
    # 真实薪资范围（蓝色线）
    ax.plot([i, i], true_salary_ranges[i], color='blue', linewidth=4, label='True salary' if i == 0 else "")
    
    # 预测薪资范围（红色线）
    ax.plot([i + 0.1, i + 0.1], pred_salary_ranges[i], color='red', linewidth=4, label='Predicted salary' if i == 0 else "")

# 添加图例和标签
ax.set_xticks([i + 0.1 for i in x])
ax.set_xticklabels([f"Job {i+1}" for i in x])
ax.set_ylabel("Salary (CNY)")
ax.set_title("True vs Predicted Salary Ranges")
ax.legend()
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 模型名称
models = [
    'BERT作为文本特征提取器+线性回归模型',
    'BERT作为文本特征提取器+XGBoost 模型',
    '微调BERT预训练模型'
]

# 对应的 MSE 值
mse_values = [0.0053, 0.0052, 0.0010]

# 创建图表
fig, ax = plt.subplots(figsize=(16, 8))
bars = ax.barh(models, mse_values, color='#279EBC')
ax.set_xlabel('均方误差 (MSE)', fontsize=28)
ax.set_title('不同模型的均方误差比较', fontsize=36)
ax.tick_params(axis='x', labelsize=28)
ax.tick_params(axis='y', labelsize=28)

# 在每个柱子上添加数值标签
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.0001, bar.get_y() + bar.get_height()/2,
            f'{width:.4f}', va='center', fontsize=28)

# 调整 x 轴的范围，以防止标签溢出
ax.set_xlim(0, max(mse_values) + 0.001)

plt.tight_layout()
plt.savefig("BERT作为文本特征提取器与微调BERT预训练模型预测效果对比.png", dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 训练轮次
epochs = list(range(1, 11))

# 模型损失值
loss_google_bert = [0.0070, 0.0037, 0.0030, 0.0025, 0.0020, 0.0017, 0.0015, 0.0013, 0.0011, 0.0010]
loss_hfl_bert = [0.0059, 0.0035, 0.0029, 0.0024, 0.0020, 0.0017, 0.0014, 0.0012, 0.0011, 0.0010]

# 创建图表
plt.figure(figsize=(12, 6))
plt.plot(epochs, loss_google_bert, marker='o', label='google-bert/bert-base-chinese', color='#1f77b4')
plt.plot(epochs, loss_hfl_bert, marker='s', label='hfl/chinese-bert-wwm-ext', color='#ff7f0e')

# 添加标题和标签
plt.title('不同基础模型的性能对比', fontsize=36)
plt.xlabel('训练轮次', fontsize=28)
plt.ylabel('损失值 (Loss)', fontsize=28)
plt.xticks(epochs, fontsize=24)
plt.yticks(fontsize=24)  # 使 x 轴标签更加清晰
plt.legend(fontsize=24)
plt.grid(True)
plt.tight_layout()

# 保存并显示图表
plt.savefig("不同基础模型的性能对比.png", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 训练轮次
epochs = list(range(1, 11))

# 模型损失值
loss_google_bert = [0.0063, 0.0035, 0.0029, 0.0024, 0.0020, 0.0017, 0.0015, 0.0013, 0.0011, 0.0010] 
loss_hfl_bert = [0.0059, 0.0035, 0.0029, 0.0024, 0.0020, 0.0017, 0.0014, 0.0012, 0.0011, 0.0010]

# 创建图表
plt.figure(figsize=(12, 6))
plt.plot(epochs, loss_google_bert, marker='o', label='对过长文本进行删除', color='#1f77b4')
plt.plot(epochs, loss_hfl_bert, marker='s', label='对过长文本进行截短', color='#ff7f0e')

# 添加标题和标签
plt.title('过长文本处理策略（删除与截断）对模型性能的影响', fontsize=36)
plt.xlabel('训练轮次', fontsize=28)
plt.ylabel('损失值 (Loss)', fontsize=28)
plt.xticks(epochs, fontsize=24)
plt.yticks(fontsize=24)  # 使 x 轴标签更加清晰
plt.legend(fontsize=24)
plt.grid(True)
plt.tight_layout()

# 保存并显示图表
plt.savefig("过长文本处理策略（删除与截断）对模型性能的影响.png", dpi=300)
plt.show()
